In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2Model.from_pretrained('distilgpt2')

model



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-5): 6 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name="distilgpt2"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForCausalLM.from_pretrained(model_name)

model




GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
prompt = "Once"
input = tokenizer(prompt, return_tensors="pt")

output = model.generate(input.input_ids, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

output_string = tokenizer.batch_decode(output)

print(output_string)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Once there to there day big. Lily was the girl went a Lily he play and play. was it was the friends Tim day in girl mom at they day. Lily.\n\n He Lily was a was a time the park, He day, the very a it so the time. and day, and he a of was all\n upon He she mom there time around a a she they the. was so the He a day" a her, " she there his wanted a friends" upon the']


In [ ]:
!pip install datasets
from datasets import load_dataset

short_stories_data=load_dataset("roneneldan/TinyStories")

short_stories_data


Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})

In [ ]:
subset_dataset=load_dataset("roneneldan/TinyStories",split="train[:1000]")
subset_dataset=subset_dataset.train_test_split(train_size=0.75)

subset_dataset

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 750
    })
    test: Dataset({
        features: ['text'],
        num_rows: 250
    })
})

In [ ]:
def preprocess_batch(batch):
  all_text_items=batch["text"]
  trimmed_text_items =[x[:500] for x in all_text_items]
  return tokenizer(trimmed_text_items)

In [ ]:
tokenized_dataset=subset_dataset.map(preprocess_batch,
                                     batched=True,
                                     batch_size=10,
                                     remove_columns=subset_dataset["train"].column_names)
tokenized_dataset

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 750
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 250
    })
})

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)



In [ ]:
!pip install transformers[torch] accelerate -U

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=10
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    data_collator=data_collator
)

tokenizer.pad_token = tokenizer.eos_token

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,7.532795
2,No log,6.394919
3,No log,6.026181
4,No log,5.768475
5,No log,5.546443
6,6.841700,5.381927
7,6.841700,5.281379


Epoch,Training Loss,Validation Loss
1,No log,7.532795
2,No log,6.394919
3,No log,6.026181
4,No log,5.768475
5,No log,5.546443
6,6.841700,5.381927
7,6.841700,5.281379
8,6.841700,5.224372
9,6.841700,5.187460
10,6.841700,5.174549


TrainOutput(global_step=940, training_loss=6.111579797623006, metrics={'train_runtime': 12886.3046, 'train_samples_per_second': 0.582, 'train_steps_per_second': 0.073, 'total_flos': 268721252794368.0, 'train_loss': 6.111579797623006, 'epoch': 10.0})

In [56]:
model=AutoModelForCausalLM.from_pretrained("./output/checkpoint-940/")
prompt = "Once upon a time there lived"
input = tokenizer(prompt, return_tensors="pt")

output = model.generate(input.input_ids, max_new_tokens=100, do_sample=True, top_k=100, top_p=0.9)

output_string = tokenizer.batch_decode(output)

print(output_string)




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Once upon a time there lived to to something girl was the time upon a girl was it were play. Lily with a mom and her girl and special day. She park, she very were upon the man.\n\n\n\n She she she decided the day. Then to the was the go to was it was it scared with playing to to many the park was to fun was little. He with it want a a happy said wanted to it littleilyily down she was playing!". � and go there like upon a']


OSError: gpt-4 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

a a a a a a a a a and a little girl a a a a a a a a a a a a a a a a a little girl a little girl a little girl a little girl 
